<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

# SAR Single Node on MovieLens (Python, CPU)

In this example, we will walk through each step of the Simple Algorithm for Recommendation (SAR) algorithm using a Python single-node implementation.

SAR is a fast, scalable, adaptive algorithm for personalized recommendations based on user transaction history. It is powered by understanding the similarity between items, and recommending similar items to those a user has an existing affinity for.

## 1 SAR algorithm

The following figure presents a high-level architecture of SAR. 

At a very high level, two intermediate matrices are created and used to generate a set of recommendation scores:

- An item similarity matrix $S$ estimates item-item relationships.
- An affinity matrix $A$ estimates user-item relationships.

Recommendation scores are then created by computing the matrix multiplication $A\times S$.

Optional steps (e.g. "time decay" and "remove seen items") are described in the details below.

<img src="https://recodatasets.z20.web.core.windows.net/images/sar_schema.svg?sanitize=true">

### 1.1 Compute item co-occurrence and item similarity

SAR defines similarity based on item-to-item co-occurrence data. Co-occurrence is defined as the number of times two items appear together for a given user. We can represent the co-occurrence of all items as a $m\times m$ matrix $C$, where $c_{i,j}$ is the number of times item $i$ occurred with item $j$, and $m$ is the total number of items.

The co-occurence matric $C$ has the following properties:

- It is symmetric, so $c_{i,j} = c_{j,i}$
- It is nonnegative: $c_{i,j} \geq 0$
- The occurrences are at least as large as the co-occurrences. I.e., the largest element for each row (and column) is on the main diagonal: $\forall(i,j) C_{i,i},C_{j,j} \geq C_{i,j}$.

Once we have a co-occurrence matrix, an item similarity matrix $S$ can be obtained by rescaling the co-occurrences according to a given metric. Options for the metric include `Jaccard`, `lift`, and `counts` (meaning no rescaling).


If $c_{ii}$ and $c_{jj}$ are the $i$th and $j$th diagonal elements of $C$, the rescaling options are:

- `Jaccard`: $s_{ij}=\frac{c_{ij}}{(c_{ii}+c_{jj}-c_{ij})}$
- `lift`: $s_{ij}=\frac{c_{ij}}{(c_{ii} \times c_{jj})}$
- `counts`: $s_{ij}=c_{ij}$

In general, using `counts` as a similarity metric favours predictability, meaning that the most popular items will be recommended most of the time. `lift` by contrast favours discoverability/serendipity: an item that is less popular overall but highly favoured by a small subset of users is more likely to be recommended. `Jaccard` is a compromise between the two.


### 1.2 Compute user affinity scores

The affinity matrix in SAR captures the strength of the relationship between each individual user and the items that user has already interacted with. SAR incorporates two factors that can impact users' affinities: 

- It can consider information about the **type** of user-item interaction through differential weighting of different events (e.g. it may weigh events in which a user rated a particular item more heavily than events in which a user viewed the item).
- It can consider information about **when** a user-item event occurred (e.g. it may discount the value of events that take place in the distant past.

Formalizing these factors produces us an expression for user-item affinity:

$$a_{ij}=\sum_k w_k \left(\frac{1}{2}\right)^{\frac{t_0-t_k}{T}} $$

where the affinity $a_{ij}$ for user $i$ and item $j$ is the weighted sum of all $k$ events involving user $i$ and item $j$. $w_k$ represents the weight of a particular event, and the power of 2 term reflects the temporally-discounted event. The $(\frac{1}{2})^n$ scaling factor causes the parameter $T$ to serve as a half-life: events $T$ units before $t_0$ will be given half the weight as those taking place at $t_0$.

Repeating this computation for all $n$ users and $m$ items results in an $n\times m$ matrix $A$. Simplifications of the above expression can be obtained by setting all the weights equal to 1 (effectively ignoring event types), or by setting the half-life parameter $T$ to infinity (ignoring transaction times).

### 1.3 Remove seen item

Optionally we remove items which have already been seen in the training set, i.e. don't recommend items which have been previously bought by the user again.

### 1.4 Top-k item calculation

The personalized recommendations for a set of users can then be obtained by multiplying the affinity matrix ($A$) by the similarity matrix ($S$). The result is a recommendation score matrix, where each row corresponds to a user, each column corresponds to an item, and each entry corresponds to a user / item pair. Higher scores correspond to more strongly recommended items.

It is worth noting that the complexity of recommending operation depends on the data size. SAR algorithm itself has $O(n^3)$ complexity. Therefore the single-node implementation is not supposed to handle large dataset in a scalable manner. Whenever one uses the algorithm, it is recommended to run with sufficiently large memory. 

## 2 SAR single-node implementation

The SAR implementation illustrated in this notebook was developed in Python, primarily with Python packages like `numpy`, `pandas`, and `scipy` which are commonly used in most of the data analytics / machine learning tasks. Details of the implementation can be found in [Recommenders/recommenders/models/sar/sar_singlenode.py](../../recommenders/models/sar/sar_singlenode.py).

## 3 SAR single-node based movie recommender

In [1]:
import sys
import logging
import scipy
import numpy as np
import pandas as pd

from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.sar import SAR
from recommenders.utils.notebook_utils import store_metadata

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"SciPy version: {scipy.__version__}")

System version: 3.8.19 (default, Mar 20 2024, 15:27:52) 
[Clang 14.0.6 ]
Pandas version: 2.0.3
NumPy version: 1.24.3
SciPy version: 1.10.1


In [2]:
from helpers import *

In [3]:
# Top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = "100k"

In [4]:
# set log level to INFO
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

### 3.1 Load Data

SAR is intended to be used on interactions with the following schema:
`<User ID>, <Item ID>, <Time>`. 

Each row represents a single interaction between a user and an item. These interactions might be different types of events on an e-commerce website, such as a user clicking to view an item, adding it to a shopping basket, following a recommendation link, and so on. 

The MovieLens dataset is well formatted interactions of Users providing Ratings to Movies (movie ratings are used as the event weight) - we will use it for the rest of the example.

In [5]:
df_articles, df_clicks, article_embeddings = load_dataset()
df_clicks = df_clicks.sample(n=500000)
df_clicks.rename(columns={'user_id': 'UserId', "click_article_id":"MovieId", "click_timestamp":"Timestamp"}, inplace=True)
df_clicks["UserId"] = df_clicks["UserId"].astype('int64')
df_clicks["Timestamp"] = df_clicks["Timestamp"].astype('int64')
df_clicks["MovieId"] = df_clicks["MovieId"].astype('int64')
df_clicks['Rating'] = 1.0
df_clicks['Rating'] = df_clicks["Rating"].astype(np.float32)

In [6]:

# print(df_clicks[['UserId','MovieId', 'Rating','Timestamp']].dtypes)
# print(data.dtypes)

In [7]:
data = df_clicks[['UserId','MovieId', 'Rating','Timestamp']].reset_index(drop=True)
data

# data = movielens.load_pandas_df(
#     size=MOVIELENS_DATA_SIZE,
#     header=["UserId", "MovieId", "Rating", "Timestamp"],
#     title_col="Title",
# )

# # Convert the float precision to 32-bit in order to reduce memory consumption
# data["Rating"] = data["Rating"].astype(np.float32)

# data.head()

,UserId,MovieId,Rating,Timestamp
0,316154,87194,1.0,1508156252374
1,65572,235920,1.0,1508161778063
2,40,157844,1.0,1507255348007
3,164378,293301,1.0,1507137874614
4,18202,233658,1.0,1507513525779
...,...,...,...,...
499995,83690,300082,1.0,1507644516221
499996,10159,96663,1.0,1506865627487
499997,91738,215861,1.0,1507241331601
499998,130075,276970,1.0,1507217199560


### 3.2 Split the data using the python random splitter provided in utilities:

We split the full dataset into a `train` and `test` dataset to evaluate performance of the algorithm against a held-out set not seen during training. Because SAR generates recommendations based on user preferences, all users that are in the test set must also exist in the training set. For this case, we can use the provided `python_stratified_split` function which holds out a percentage (in this case 25%) of items from each user, but ensures all users are in both `train` and `test` datasets. Other options are available in the `dataset.python_splitters` module which provide more control over how the split occurs.


In [8]:
header = {
    "col_user": "UserId",
    "col_item": "MovieId",
    "col_rating": "Rating",
    "col_timestamp": "Timestamp",
    "col_prediction": "Prediction",
}

In [9]:
train, test = python_stratified_split(
    data, ratio=0.75, col_user=header["col_user"], col_item=header["col_item"], seed=42
)


In this case, for the illustration purpose, the following parameter values are used:

|Parameter|Value|Description|
|---------|---------|-------------|
|`similarity_type`|`jaccard`|Method used to calculate item similarity.|
|`time_decay_coefficient`|30|Period in days (term of $T$ shown in the formula of Section 1.2)|
|`time_now`|`None`|Time decay reference.|
|`timedecay_formula`|`True`|Whether time decay formula is used.|

In [10]:
model = SAR(
    similarity_type="jaccard", 
    time_decay_coefficient=30, 
    time_now=None, 
    timedecay_formula=True, 
    **header
)

In [11]:
model.fit(train)

2024-05-26 23:40:44 INFO     Collecting user affinity matrix
2024-05-26 23:40:44 INFO     Calculating time-decayed affinities
2024-05-26 23:40:44 INFO     Creating index columns
2024-05-26 23:40:44 INFO     Building user affinity sparse matrix
2024-05-26 23:40:44 INFO     Calculating item co-occurrence
2024-05-26 23:40:44 INFO     Calculating item similarity
2024-05-26 23:40:44 INFO     Using jaccard based similarity
2024-05-26 23:40:48 INFO     Done training


In [12]:
top_k = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

2024-05-26 23:40:48 INFO     Calculating recommendation scores
2024-05-26 23:40:50 INFO     Removing seen items
/Users/m/dev/6be709c0/oc-ai-engineer/OC/P10/venv/lib/python3.8/site-packages/scipy/sparse/_data.py:124: RuntimeWarning: invalid value encountered in multiply
  return self._with_data(self.data * other)


The final output from the `recommend_k_items` method generates recommendation scores for each user-item pair, which are shown as follows.

In [13]:
top_k_with_titles = top_k.join(
    data[["MovieId"]].drop_duplicates().set_index("MovieId"),
    on="MovieId",
    how="inner",
).sort_values(by=["UserId", "Prediction"], ascending=False)

top_k_with_titles.head(10)

,UserId,MovieId,Prediction
583700,322666,331531,1.438075e-186
583709,322666,113993,1.115625e-186
583703,322666,361939,1.115625e-186
583702,322666,97141,1.115625e-186
583708,322666,309520,1.115625e-186
583704,322666,86266,1.115625e-186
583706,322666,309504,1.115625e-186
583701,322666,311661,1.115625e-186
583705,322666,88961,1.115625e-186
583707,322666,314518,1.115625e-186


### 3.3 Evaluate the results

It should be known that the recommendation scores generated by multiplying the item similarity matrix $S$ and the user affinity matrix $A$ **DOES NOT** have the same scale with the original explicit ratings in the movielens dataset. That is to say, SAR algorithm is meant for the task of *recommending relevent items to users* rather than *predicting explicit ratings for user-item pairs*. 

To this end, ranking metrics like precision@k, recall@k, etc., are more applicable to evaluate SAR algorithm. The following illustrates how to evaluate SAR model by using the evaluation functions provided in Recommenders library.

In [17]:
test

,UserId,MovieId,Rating,Timestamp
62405,1,160474,1.0,1506879426845
70434,5,348081,1.0,1507807219692
116445,5,161801,1.0,1507560379155
256290,5,36399,1.0,1507721482747
126920,5,202763,1.0,1507387973525
...,...,...,...,...
54708,321743,294111,1.0,1508197476450
321184,321984,315104,1.0,1508199012501
401039,321991,158158,1.0,1508199171012
368872,322522,234308,1.0,1508206163271


In [16]:
top_k

,UserId,MovieId,Prediction
0,1,207975,6.511537e-188
1,1,195177,5.803762e-188
2,1,224730,5.005744e-188
3,1,100153,4.883653e-188
4,1,99935,4.883653e-188
...,...,...,...
583705,322666,88961,1.115625e-186
583706,322666,309504,1.115625e-186
583707,322666,314518,1.115625e-186
583708,322666,309520,1.115625e-186


In [14]:
# all ranking metrics have the same arguments
args = [test, top_k]
kwargs = dict(
    col_user="UserId",
    col_item="MovieId",
    col_rating="Rating",
    col_prediction="Prediction",
    relevancy_method="top_k",
    k=TOP_K,
)

eval_map = map_at_k(*args, **kwargs)
eval_ndcg = ndcg_at_k(*args, **kwargs)
eval_precision = precision_at_k(*args, **kwargs)
eval_recall = recall_at_k(*args, **kwargs)

TypeError: ndcg_at_k() got an unexpected keyword argument 'col_user'

In [ ]:
print(f"Model:",
      f"Top K:\t\t {TOP_K}",
      f"MAP:\t\t {eval_map:f}",
      f"NDCG:\t\t {eval_ndcg:f}",
      f"Precision@K:\t {eval_precision:f}",
      f"Recall@K:\t {eval_recall:f}", sep='\n')

In [ ]:
# Record results for tests - ignore this cell
store_metadata("map", eval_map)
store_metadata("ndcg", eval_ndcg)
store_metadata("precision", eval_precision)
store_metadata("recall", eval_recall)

## References
Note SAR is a combinational algorithm that implements different industry heuristics. The followings are references that may be helpful in understanding the SAR logic and implementation. 

1. Badrul Sarwar, *et al*, "Item-based collaborative filtering recommendation algorithms", WWW, 2001.
2. Scipy (sparse matrix), url: https://docs.scipy.org/doc/scipy/reference/sparse.html
3. Asela Gunawardana and Guy Shani, "A survey of accuracy evaluation metrics of recommendation tasks", The Journal of Machine Learning Research, vol. 10, pp 2935-2962, 2009.	